<h2 id="Introduction">Introduction<h2>

<h5>Countdown is a numbers game whereby contestants are given a selection of numbers and a target number. Using only basic arithmetic operations (addition, subtraction, multiplication, and division), the contestants must try to reach the target number using only the chosen numbers. This project, done in a Jupyter Notebook, explains the game in detail and contains python code of the game being completed by solve_numbers function. The goal is to have a working version of the countdown game.

<h2 id="Explanation">Explanation<h2>

<h5>
Players will have a time limit of between 30 to 60 seconds to solve this puzzle. Whatever player has the closest number to the target number will win the round. 

How the game works: 

- Players are given a selection of six numbers, usually a mix of small numbers (1-10) and large numbers (25, 50, 75, or 100).

- A three-digit target number is randomly generated. Generally starts at 101 to 999. 

- Players have to use some or all of the six numbers (using each number once and only once) to reach the target number exactly, or as close as possible.

- They can use any combination of addition, subtraction, multiplication, and division to manipulate the numbers. However at no point can the number be a fraction or decimal. Subtraction may only be used if the result is a positive number.



<h2 id="Binomial_Coefficient_Formula">Binomial Coefficient Formula<h2>

<h5>
Although the algorithm below doesn't use this formula directly, it does have many similarities. This is similar to selecting a subset of numbers from a larger set, which is essentially what combinations represent in combinatorics.

- Combination of Numbers: Generates all combinations of numbers from the list. This is similar to selecting a subset of numbers from a larger set, which is essentially what combinations represent in combinatorics.

- Permutations of Numbers: For each combination of numbers, it calculates the number of permutations of those numbers. This is similar to the concept of arranging items in a specific order, which is what permutations represent. In this case, the order matters because changing the order of numbers in an expression changes the result.

- Permutations of Operators: For each combination of numbers, it also calculates the number of permutations of operators. Since there are four basic arithmetic operators and they can be repeated, the number of permutations of operators is calculated as 4𝑟−1 where 𝑟 is the number of numbers in the combination. This is similar to the idea of selecting a subset of items from a set with replacement, which is a concept related to permutations.

- Total Count: The total number of valid expressions is calculated by multiplying the number of permutations of numbers by the number of permutations of operators for each combination of numbers. This multiplication is reminiscent of the multiplication principle in combinatorics, where the total count of outcomes is calculated by multiplying the counts of individual steps or choices.

In [93]:
from itertools import combinations, permutations
import math

def valid_expressions(numbers):
    total_count = 0
    
    # Generate all combinations of numbers
    for r in range(1, len(numbers) + 1):
        for number_combination in combinations(numbers, r):
            num_count = math.factorial(r)  # Number of permutations of numbers
            op_count = 4 ** (r - 1)         # Number of permutations of operators
            total_count += num_count * op_count
    
    return total_count

# Example
numbers = [1, 9, 6, 7, 50, 100]
total_expressions = valid_expressions(numbers)
print("Total valid expressions:", total_expressions)


Total valid expressions: 946686


<h2 id="Calculating_For_a_target_Number">Calculating For a Target Number<h2>

<h5>
The algorithm below finds the total valid expressions for a specified target number, although it does contain expression that do not comply with rules, i.e. the use of negative numbers. 

In [97]:
from itertools import combinations, permutations, product

def find_valid_expressions(numbers, target):
    valid_expressions = []

    # Open the file for writing due to a large possible output
    with open("results.txt", "w") as file:
        # Generate all combinations of numbers
        for r in range(1, len(numbers) + 1):
            for number_combination in combinations(numbers, r):
                # Generate all permutations of numbers
                for num_permutation in permutations(number_combination):
                    # Generate all permutations of operators
                    for op_permutation in product('+-*/', repeat=r - 1):
                        expression = ''.join(str(num) + op for num, op in zip(num_permutation, op_permutation))
                        expression += str(num_permutation[-1])  # Add the last number
                        # Evaluate the expression
                        try:
                            result = eval(expression)
                            if result == target:
                                valid_expressions.append(expression)
                                file.write("Valid expression: {}\n".format(expression))
                        except ZeroDivisionError:
                            pass  # Skip division by zero errors

    return valid_expressions

# Example usage:
numbers = [2, 3, 5, 7, 25, 100]
target = 725
valid_expressions = find_valid_expressions(numbers, target)
print("Total valid expressions for target", target, ":", len(valid_expressions))


Total valid expressions for target 725 : 478


<h2>

<h5>
Creating expressions that only use whole numbers and does not use negative numbers in their calculations, this tends to be slow but accurate.

In [155]:
from itertools import combinations, permutations, product

def evaluate_expression(num_permutation, op_permutation):
    # Combine numbers and operators to form the expression
    expression = ''.join(str(num) + op for num, op in zip(num_permutation, op_permutation))
    expression += str(num_permutation[-1])  # Add the last number

    # Evaluate the expression while checking for negative intermediate results
    nums = num_permutation + (num_permutation[-1],)  # Include last number
    result = nums[0]
    for i in range(len(op_permutation)):
        op = op_permutation[i]
        next_num = nums[i + 1]
        if op == '+':
            result += next_num
        elif op == '-':
            result -= next_num
            if result < 0:  # Check for negative intermediate result
                return None  # Invalid expression due to negative result
        elif op == '*':
            result *= next_num
        elif op == '/':
            if next_num == 0 or result % next_num != 0:
                return None  # Invalid expression due to division by zero or non-integer result
            result //= next_num  # Use integer division

    return result

def find_valid_expressions(numbers, target):
    valid_expressions = []

    # Open the file for writing due to a large possible output
    with open("results.txt", "w") as file:
        # Generate all combinations of numbers
        for r in range(1, len(numbers) + 1):
            for number_combination in combinations(numbers, r):
                # Generate all permutations of numbers
                for num_permutation in permutations(number_combination):
                    # Generate all permutations of operators
                    for op_permutation in product('+-*/', repeat=r - 1):
                        # Evaluate the expression and check for validity
                        result = evaluate_expression(num_permutation, op_permutation)
                        if result is not None and result == target:
                            expression = ''.join(str(num) + op for num, op in zip(num_permutation, op_permutation))
                            expression += str(num_permutation[-1])  # Add the last number
                            valid_expressions.append(expression)
                            file.write("Valid expression: {}\n".format(expression))

    return valid_expressions

import random

def generate_numbers():
    # Define the small numbers (1 to 10)
    small_numbers = list(range(1, 11))
    # Define the large numbers (25, 50, 75, 100)
    large_numbers = [25, 50, 75, 100]

    # Randomly decide how many big numbers and small numbers to generate
    num_big_numbers = random.randint(0, 4)  # Can select 0 to 4 big numbers
    num_small_numbers = 6 - num_big_numbers  # Remaining slots are for small numbers

    # Randomly select big numbers and small numbers
    selected_numbers = random.sample(small_numbers, num_small_numbers) + \
                       random.sample(large_numbers, num_big_numbers)

    return selected_numbers

# Generate 6 number tiles
numbers = generate_numbers()
print("Generated Numbers: ", numbers)

target = random.randint(101, 999)
print("Target: ", target)

# Testing to see if it still runs when impossible
# testNumbers = [2, 4, 6, 8, 10, 2]
# testTarget = 999

valid_expressions = find_valid_expressions(numbers, target)
print("Total valid expressions for target", target, ":", len(valid_expressions))


Generated Numbers:  [9, 6, 3, 7, 1, 5]
Target:  135
Total valid expressions for target 135 : 1179


## First Attempt

****

In [156]:
from itertools import permutations, product

def solve_numbers(numbers, target):
    def evaluate(expression):
        try:
            return int(eval(expression))
        except ZeroDivisionError:
            return None

    def generate_expressions(nums, current_expr='', index=0):
        if index == len(nums):
            result = evaluate(current_expr)
            if result == target:
                return current_expr
            return None

        result = None
        num = nums[index]
        new_index = index + 1
        result = result or generate_expressions(nums, current_expr + str(num), new_index)
        result = result or generate_expressions(nums, current_expr + '+' + str(num), new_index)
        result = result or generate_expressions(nums, current_expr + '-' + str(num), new_index)
        result = result or generate_expressions(nums, current_expr + '*' + str(num), new_index)
        if num != 0:
            result = result or generate_expressions(nums, current_expr + '/' + str(num), new_index)
        return result

    result = generate_expressions(numbers)
    if result:
        return result + " = " + str(target)
    else:
        return None

# Example usage:
numbers = [1, 3, 5, 7, 25, 100]
target = 142
print(solve_numbers(numbers, target))


135+725/100 = 142


<h2 id="References">References<h2>

<h4>
<a href="http://datagenetics.com/blog/august32014/index.html">Brute Force Approach</a>
<br>
<a href="https://en.wikipedia.org/wiki/Countdown_(game_show)">Countdown Information</a>
<br>
<a href="https://www.youtube.com/watch?app=desktop&v=EKN51vLKves">Binomial Coefficient Formula</a>